In [2]:
import os
import pandas as pd
import numpy as np
from scipy import misc
from sklearn.cross_validation import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
np.random.seed(31415) # for reproducibility

legend = pd.read_csv("/root/sharedfolder/legend/legend-clean.csv")
legend_aug = pd.read_csv("/root/sharedfolder/legend/legend-aug.csv")
aug_path = "/root/sharedfolder/images-aug/"

legend['emotion'] = legend['emotion'].str.lower()

# 'neutral' = 0
# 'happiness' = 1
# 'surprise' = 2
# 'anger' = 3
# 'sadness' = 4
# 'contempt' = 5
# 'disgust' = 6
# 'fear' = 7
legend.replace('neutral', 0, inplace=True)
legend.replace('happiness', 1, inplace=True)
legend.replace('surprise', 2, inplace=True)
legend.replace('anger', 3, inplace=True)
legend.replace('sadness', 4, inplace=True)
legend.replace('contempt', 5, inplace=True)
legend.replace('disgust', 6, inplace=True)
legend.replace('fear', 7, inplace=True)
legend_aug.replace('surprise', 2, inplace=True)
legend_aug.replace('anger', 3, inplace=True)
legend_aug.replace('sadness', 4, inplace=True)
legend_aug.replace('contempt', 5, inplace=True)
legend_aug.replace('disgust', 6, inplace=True)
legend_aug.replace('fear', 7, inplace=True)

frames = [legend, legend_aug]
new_legend = pd.concat(frames)

# convert from secondary emotions to primary emotions
new_legend.replace(5, 3, inplace=True)
new_legend.replace(6, 3, inplace=True)
new_legend.replace(2, 7, inplace=True)

# convert 'fear' = 7 to 'fear' = 2 to maintain sequence
# primary emotions
# 'neutral' = 0
# 'happiness' = 1
# 'fear' = 2
# 'anger' = 3
# 'sadness' = 4
new_legend.replace(7, 2, inplace=True)

new_legend['emotion'].unique()

array([0, 1, 2, 3, 4])

In [3]:
batch_size = 128
nb_classes = 5
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 96, 96
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (5, 5)
# convolution kernel size
kernel_size = (10, 10)

In [4]:
# convert images to numpy.ndarray
images = np.empty(shape=[len(new_legend), img_rows, img_cols])
for i in range(0,len(new_legend)):
    temp = aug_path + new_legend.iloc[i,0]
    img = misc.imread(temp, flatten=True,)
    img = misc.imresize(img, (img_rows, img_cols))
    images[i] = img    

In [5]:
emotions = new_legend['emotion']

In [6]:
# the data, shuffled and split between train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, emotions, test_size=0.2)

In [7]:
X_train.shape

(28928, 96, 96)

In [8]:
X_test.shape

(7233, 96, 96)

In [9]:
img_rows, img_cols = X_train.shape[1], X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)

In [10]:
X_train.shape

(28928, 1, 96, 96)

In [11]:
X_test.shape

(7233, 1, 96, 96)

In [12]:
input_shape = (1, img_rows, img_cols)

In [13]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

('X_train shape:', (28928, 1, 96, 96))
(28928, 'train samples')
(7233, 'test samples')
Train on 28928 samples, validate on 7233 samples
Epoch 1/12
28928/28928 [==============================] - 9876s - loss: 1.2416 - acc: 0.4652 - val_loss: 0.9478 - val_acc: 0.6118
Epoch 2/12
28928/28928 [==============================] - 9762s - loss: 0.9588 - acc: 0.6034 - val_loss: 0.8243 - val_acc: 0.6621
Epoch 3/12
28928/28928 [==============================] - 9856s - loss: 0.8370 - acc: 0.6585 - val_loss: 0.7645 - val_acc: 0.6891
Epoch 4/12
28928/28928 [==============================] - 9770s - loss: 0.7566 - acc: 0.6956 - val_loss: 0.6854 - val_acc: 0.7153
Epoch 5/12
28928/28928 [==============================] - 9614s - loss: 0.6901 - acc: 0.7222 - val_loss: 0.6263 - val_acc: 0.7419
Epoch 6/12
28928/28928 [==============================] - 9637s - loss: 0.6432 - acc: 0.7445 - val_loss: 0.5798 - val_acc: 0.7704
Epoch 7/12
28928/28928 [==============================] - 9519s - loss: 0.6023 - acc